# Table of Contents
1. [Things to Do](#Things-to-Do)
1. [Introduction](#Introduction)
1. [Set up Python](#Set-up-Python)
2. [Functions](#Functions)
3. [Getting Data](#Get-Data)
4. [Analyze Data](#Analyze-Data):
    - [Population Size](#Population-Size) - noticeable changes
    - [Sex Distribution](#Sex-Distribution) - does not change
    - [Tail Condition Distribution](#Tail-Condition-Distribution)- noticeable change
        - [Severity of Autotomy](#Severity-of-Autotomy)- noticeable change
    - [Location](#Location)
    - [Morphometrics](#Morphometrics):
        - [SVL](#SVL)
        - [TL](#TL)
        - [RTL](#RTL)
        - [Mass](#Mass)- noticeable change (Check Males)
    - [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
    - [Captures](#Captures)
    - [Growth](#Growth)
        - [SVL Growth](#SVL-Growth)
        - [TL Growth](#TL-Growth)
        - [RTL Growth](#RTL-Growth)
        - [Mass Growth](#Mass-Growth)
    - [Correlations to Population](#Correlations-to-Population)
5. [Export Files](#Export-Files)

# Things to Do

Also search for "to be done"

## [Resume Here](#resume)

## Introduction

This notebook contains code and output of descriptive analyses for the 2000-2017 CC dataset after cleaning.

The objectives of this notebook are to describe the community of the _Sceloporus jarrovii_ adults and _Sceloporus virgatus_ lizards in the Crystal Creek wash from 2000 until 2017.  The population demographic metrics we examine are: [population size](#Population-Size), [sex distribution](#Sex-Distribution), [tail condition distribution](#Tail-Condition-Distribution), [location](#Location), [morphometrics](#Morphometrics) -- [SVL](#SVL), [TL](#TL), [RTL](#RTL), [mass](#Mass) --,  [survival and rates and likelihood of recapture](#Survival-and-Rates-and-Likelihood-of-Recapture), and [growth](#Growth).

We will examine these metrics and interactions among them with particular interest in the impact of environmental factors from year to year.


##  Set up Python

First we will need to set up the python environment, importing the necessary packages and setting the display options.

[Top](#Table-of-Contents)

In [1]:
import pandas as pd
import numpy as np
import pingouin as pg
import os, glob, logging
from summary_functions import *
from scipy import stats
from monthlit import *
from prettyprint import *


import plotly
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
# plotly.tools.set_config_file(world_readable=True)


# increase print limit
pd.options.display.max_rows = 99999
pd.options.display.max_columns = 50

### Setting File Locations

In [2]:
deviceDict = {'dataBig':{'source':'S:/Chris/TailDemography/TailDemography/outputFiles'
                         ,'log':'S:/Chris/TailDemography/TailDemography/Scripts and notes/Descriptive/'
                         ,'output':'S:/Chris/TailDemography/TailDemography/outputFiles/'},
              'silverSurfer':{'source':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles'
                              ,'log':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                              ,'output':'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\outputFiles\\Descriptive\\'}
              ,'dataPers':{'source':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:\\Users\\craga_eowcrpe\\Google Drive\\TailDemography\\Scripts and notes\\Descriptive\\'
                           ,'output':'C:/Users/Christopher/Google Drive/TailDemography/outputFiles/Descriptve/'}
             ,'gandolf':{'source':'C:/Users/craga/Google Drive/TailDemography/outputFiles'
                           ,'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/'
                           ,'output':'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}}

### Choose Device

In [3]:
device = deviceDict['gandolf']
device

{'source': 'C:/Users/craga/Google Drive/TailDemography/outputFiles',
 'log': 'C:/Users/craga/Google Drive/TailDemography/Scripts and notes/Descriptive/',
 'output': 'C:/Users/craga/Google Drive/TailDemography/outputFiles/Descriptive/'}

# Source Data


### Logging

In [4]:
logging.basicConfig(filename=device['log']+'Desriptive Analyses.log'
                    , filemode='a',
                    format='%(funcName)s - %(levelname)s - %(message)s - %(asctime)s', level=logging.DEBUG)

## Functions

This section contains functions that were created for this notebook.

- [distribution](#distribution) #delete this we will use scipy stats describe instead
- [monthlit](#monthlit)
- [description](#description)
- [vocab_run](#vocab_run)

### distribution
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*distribution* takes a series or list of numeric objects, *x*, and returns descriptive stats of x including
        n, minimum, maximum, median, sIQR, mean, and stdev
    
Here are a few examples of how *distribution* works.

In [5]:
foo = [0,1,2,'r']
distribution(foo)

In [6]:
bar = [0,1,2]
distribution(bar)

,n,minimum,maximum,median,siqr,mean,stdev
0,3,0,2,1.0,0.5,1.0,1.0


[Back to Functions](#Functions)

## monthlit
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

Here are a few examples of how _monthlit_ works.

In [7]:
dates = pd.DataFrame(data={'dates':['2018-12-9','2019-8-5', '2017/7/4',np.nan,None]})
dates.dates = pd.to_datetime(dates.dates)
dates

,dates
0,2018-12-09
1,2019-08-05
2,2017-07-04
3,NaT
4,NaT


In [8]:
np.isnan(np.nan)

True

In [9]:
monthlit(dates.dates.dt.month[0])

'Dec'

In [10]:
dates.dates.dt.month.apply(monthlit)

0    Dec
1    Aug
2    Jul
3    NaN
4    NaN
Name: dates, dtype: object

[Back to Functions](#Functions)

## description
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

In [11]:
def description(x,variable,percentage=False):
    if percentage:
            res = x[variable].describe()
            res[['mean','std','min','25%','50%','75%','max']] = res[['mean','std','min','25%','50%','75%','max']]\
            .apply(lambda x:x*100) 
#Need to Add CI calculation to this function
#             meanCI = 'not calculated'
    else:
        res = x[variable].describe() 
    res['siqr'] = (res['75%']-res['25%'])/2
    res['meanCI'] = 'not calculated'
    return res

### vocab_run
[Back to Top](#Table-of-Contents)

[Back to Functions](#Functions)

*vocab_run* takes a list, joins its the first the elements with a separator placing a different separator between
     the penultimate and final members of the list and returns the result as a string
     :param x: a list of strings to be concatenated
     :param connector_dict: a dictionary with keys describing the size of the list and values indicating the type of
     connectors separate the list elements.
    
Here are a few examples of how *vocab_run* works.

In [12]:
print("Could you bring some {} please?".format(vocab_run(['foo','bar','stuffkins'])))

Could you bring some foo, bar and stuffkins please?


In [13]:
print("You can either have {}.  You'll have to make a choice."\
      .format(vocab_run(['foo','bar','stuffkins'],connector_dict={1: None, 2: ' or ', 'run': ', '})))

You can either have foo, bar or stuffkins.  You'll have to make a choice.


[Back to Functions](#Functions)

We'll display all files in the source folder with the prefix _'cleaned CC data 2000-2017'_. The file names will be saved in a variable, _mysourcefiles_.

## Get Data
[Top](#Table-of-Contents)

Here we can set the locations from which we get data and to which we export it.

In [14]:
os.chdir(device['source'])
mysourcefiles = glob.glob('cleaned CC data 2000-2017_*.csv')
mysourcefiles

['cleaned CC data 2000-2017_2019-01-31 01hrs43min.csv',
 'cleaned CC data 2000-2017_2019-03-10 14hrs42min.csv',
 'cleaned CC data 2000-2017_2019-03-12 21hrs48min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs52min.csv',
 'cleaned CC data 2000-2017_2019-03-12 22hrs55min.csv',
 'cleaned CC data 2000-2017_2019-04-25 00hrs58min.csv',
 'cleaned CC data 2000-2017_2019-04-25 01hrs00min.csv',
 'cleaned CC data 2000-2017_2019-05-02 00hrs17min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs03min.csv',
 'cleaned CC data 2000-2017_2019-05-02 01hrs08min.csv',
 'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [15]:
pd.to_datetime(mysourcefiles[0].split("_")[1].split(".csv")[0].split(' ')[0])

Timestamp('2019-01-31 00:00:00')

Automate getting the latest file

In [16]:
[latestFile for latestFile in mysourcefiles if \
 max({pd.to_datetime(afile.split("_")[1].split(".csv")[0].split(' ')[0]) \
      for afile in mysourcefiles}) == pd.to_datetime(latestFile.split("_")\
                                                     [1].split(".csv")[0].split(' ')[0])]

['cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv']

In [17]:
min({afile.split(' ')[-1].replace('hrs','').replace('min.csv','') for afile in mysourcefiles})

'0017'

In [18]:
latest = mysourcefiles[-1]
latest

'cleaned CC data 2000-2017_2019-05-04 00hrs33min.csv'

The most recent file is the one we will use as _df_ in our descriptive analysis.

In [19]:
df=pd.read_csv(latest)
df.date=pd.to_datetime(df.date)
df['month']=df.date.dt.month.apply(monthlit)
df = df.loc[df.svl>=56]
df.head()

,species,toes_orig,sex,date,svl,tl,rtl,autotomized,mass,location,meters,newRecap,painted,sighting,paint.mark,vial,misc,rtl_orig,toes,toe_pattern,description,action,pattern_b,replacement,year,tl_svl,mass_svl,all_meters,initialCaptureDate,year_diff,smallest_svl,svl_diff,liznumber,sex_count,daysSinceCapture,capture,month
0,j,3-7-11-19,m,2002-07-14,63.0,92.0,0.0,False,10.0,halfway up to site,-200,N,NaN,NaN,b101t,toes in vial 58-02,NaN,0.0,3-7-11-19,NaN,NaN,NaN,NaN,NaN,2002,1.460317,0.158730,['-200'],2002-07-14,0,63.0,0.0,375,1,0,1,Jul
1,j,3-7-11-18,m,2002-07-14,66.0,92.0,0.0,False,10.8,left downstream 100m v 1 falls,-100,N,NaN,NaN,b102t,toes in vial 59-02,NaN,0.0,3-7-11-18,NaN,NaN,NaN,NaN,NaN,2002,1.393939,0.163636,['-100' '87'],2002-07-14,0,66.0,0.0,374,1,0,1,Jul
2,j,3-7-12-16,m,2002-07-14,68.0,103.0,0.0,False,10.3,90m v 1 falls,-90,N,NaN,NaN,b103t,toes in vial 60-02,NaN,0.0,3-7-12-16,NaN,NaN,NaN,NaN,NaN,2002,1.514706,0.151471,['-90'],2002-07-14,0,68.0,0.0,376,1,0,1,Jul
3,j,10,m,2002-07-14,85.0,118.0,0.0,False,19.5,sb - trail intersection v 1 falls,-20,R,toe loss may be natural,NaN,b104t,NaN,NaN,0.0,10,NaN,NaN,NaN,NaN,NaN,2002,1.388235,0.229412,['-20' '-12' '-35'],2002-07-14,0,85.0,0.0,830,2,0,1,Jul
4,v,10-16,f,2002-07-03,63.0,49.0,21.0,True,11.6,left side @ base juniper 8 m ^ sb; 15m v 1 falls,-15,N,painted; gravid; <5 mites in pockets,NaN,w1a,toes in vial 34-02 (10-16),NaN,21.0,10-16,NaN,NaN,NaN,NaN,NaN,2002,0.777778,0.184127,['-15'],2002-07-03,0,63.0,0.0,1414,1,0,1,Jul


## Analyze Data
[Top](#Table-of-Contents)

We will first examine the range and distribution of number of variables in our data set:
- [Population Size](#Population-Size)
- [Species Distribution](#Species-Distribution)
- [Sex Distribution](#Sex-Distribution)
- [Tail Condition Distribution](#Tail-Condition-Distribution)
- [Severity of Autotomy](#Severity-of-Autotomy)
- [Location](#Location)
- [Morphometrics](#Morphometrics):
    - [SVL](#SVL)
    - [TL](#TL)
    - [RTL](#RTL)
    - [Mass](#Mass)
- [Survival and Rates and Likelihood of Recapture](#Survival-and-Rates-and-Likelihood-of-Recapture)
- [Captures](#Captures)
- [Growth](#Growth)
    - [SVL Growth](#SVL-Growth)
    - [TL Growth](#TL-Growth)
    - [RTL Growth](#RTL-Growth)
    - [NTL Growth](#NTL-Growth)
    - [Mass Growth](#Mass-Growth)
We will use first captures of each lizard in a year for these analyses.
- [Correlations to Population](#Correlations-to-Population)

## Reducing the analyses sample by date range and capture

In [20]:
monthsExcluded = ['Oct','Dec']
idx_exclusion = (df.month.isin(monthsExcluded))&(df.capture==1)&(df.species!='j')
print("The number of individuals captured for the first time in {} is {}. \
These are excluded for further analyses.".format(vocab_run(monthsExcluded,{1: None, 2: ' or ', 'run': ', '}),
                                                df.loc[idx_exclusion].liznumber.nunique()))
df=df.loc[~idx_exclusion]
df=df.loc[(df.species=='j')& (df.sex.isin(['m','f']))]

The number of individuals captured for the first time in Oct or Dec is 0. These are excluded for further analyses.


Here we create datasets including only the first or last sighting in each year for a given animal.

In [21]:
df_lastInYear = df.loc[~df.duplicated(subset=['liznumber','year'],keep='last')]
df_firstInYear = df.loc[~df.duplicated(subset=['liznumber','year'])]

### Population Size

We will begin by examining the range and distribution of individuals in the population.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Add a column of counts of lizards weighted by the person-hours for the year, or provide argument based on asymptote analysis that we captured all lizards in the site.

In [22]:
populationSize = df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()
populationSize['percChange'] = -(1-populationSize.liznumber/populationSize.liznumber.shift())
populationSize

,year,liznumber,percChange
0,2000,124,NaN
1,2001,92,-0.258065
2,2002,97,0.054348
3,2003,64,-0.340206
4,2004,55,-0.140625
5,2005,60,0.090909
6,2006,60,-0.000000
7,2007,51,-0.150000
8,2008,67,0.313725
9,2009,54,-0.194030


 For some reason the first trace dissappears after I add the second one.  Need to fix this

In [23]:
popSize = go.Scatter(x = populationSize.year
           , y = populationSize.liznumber
          ,name = 'population size')
percentChange = go.Scatter(x = populationSize.year
           , yaxis = 'y2', y = populationSize.percChange
          ,name = 'percent change')
data = [popSize]#,percentChange]
layout = go.Layout(
    title = 'Population Size for Sceloporus jarrovii adults 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero'
        ),
    yaxis2 = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18),
        rangemode = 'tozero',
        side = 'right'
        )
)
fig = go.Figure(
        data = data,
        layout = layout)
plot(fig, filename = 'Population Size for Sceloporus jarrovii adults.html')
iplot(fig, filename = 'Population Size for Sceloporus jarrovii adults.html')
# pio.to_image(fig, format='html')

We see a large decline in the <i>S. jarrovii</i> population size.

In [24]:
description(populationSize,'liznumber')

count                 18
mean             61.1667
std              22.3771
min                   32
25%                   48
50%                 54.5
75%                63.25
max                  124
siqr               7.625
meanCI    not calculated
Name: liznumber, dtype: object

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Sex Distribution

We will begin by examining the range and distribution of _sex_ values.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Maybe aymptote analysis should be extended to examine asymptote by demographic factors as well.

In [25]:
populationSize_sex = df_firstInYear.groupby(['year','sex']).liznumber.nunique().reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'}),
       on=['year'])
populationSize_sex\
.loc[populationSize_sex.sex=='m','propMale'] = populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='m'].liznumberYear
populationSize_sex\
.loc[populationSize_sex.sex=='f','propFemale'] = (populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumber/populationSize_sex\
.loc[populationSize_sex.sex=='f'].liznumberYear)
populationSize_sex.to_csv(device['output']+'population size 56 plus.csv',index = False)
populationSize_sex

,year,sex,liznumber,liznumberYear,propMale,propFemale
0,2000,f,65,124,NaN,0.524194
1,2000,m,59,124,0.475806,NaN
2,2001,f,47,92,NaN,0.510870
3,2001,m,45,92,0.489130,NaN
4,2002,f,51,97,NaN,0.525773
5,2002,m,46,97,0.474227,NaN
6,2003,f,35,64,NaN,0.546875
7,2003,m,29,64,0.453125,NaN
8,2004,f,30,55,NaN,0.545455
9,2004,m,25,55,0.454545,NaN


In [26]:
Sjarrovii = go.Scatter(x = populationSize_sex.loc[(populationSize_sex.propMale.notna())].year
           , y = populationSize_sex.loc[(populationSize_sex.propMale.notna())].propMale
          ,name = '<i>S. jarrovii</i>')
data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion of Males by Year for <i>Sceloporus jarrovii</i> adults 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat = ".2%",
        title = 'Percentage of Males',
        titlefont = dict(
            size = 18),
    range = [0,1])
    
)
fig = go.Figure(
        data = data,
        layout = layout)
plot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii adults 2000-2017 56 plus.html')
iplot(fig, filename = 'Proportion of Males by Species and Year for Sceloporus jarrovii adults 2000-2017 56 plus.html')

In [27]:
description(populationSize_sex,'propMale',True)[['mean','std','min','25%','50%','75%','max','siqr']]

mean    45.6608
std     4.14294
min     34.8837
25%     43.7145
50%     46.4815
75%     48.1903
max     51.6667
siqr     2.2379
Name: propMale, dtype: object

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

To be Done

There is not much variation in the sex ratio in the population here, suggesting that these drops do not differentially  impact males or females overall.  We should confirm that there isn't a difference in particular age/size groups.  Such a difference might be explained by developmental or social changes which leave certain groups more vulnerable than others.

### Tail Condition Distribution

Here we look at the proportion of individuals in our data who have experienced autotomy.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [28]:
populationSize_aut = df_firstInYear.groupby(['year','autotomized']).liznumber.nunique()\
.reset_index()\
.merge(df_firstInYear.groupby(['year']).liznumber.nunique().reset_index()\
       .rename(columns={'liznumber':'liznumberYear'})
       ,on=['year'])
populationSize_aut\
.loc[populationSize_aut.autotomized,'propAutotomized'] = populationSize_aut\
.loc[populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[populationSize_aut.autotomized].liznumberYear
populationSize_aut\
.loc[~populationSize_aut.autotomized,'propIntact'] = (populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumber/populationSize_aut\
.loc[~populationSize_aut.autotomized].liznumberYear)
populationSize_aut

,year,autotomized,liznumber,liznumberYear,propAutotomized,propIntact
0,2000,False,79,124,NaN,0.637097
1,2000,True,45,124,0.362903,NaN
2,2001,False,58,92,NaN,0.630435
3,2001,True,34,92,0.369565,NaN
4,2002,False,67,97,NaN,0.690722
5,2002,True,30,97,0.309278,NaN
6,2003,False,42,64,NaN,0.656250
7,2003,True,22,64,0.343750,NaN
8,2004,False,35,55,NaN,0.636364
9,2004,True,20,55,0.363636,NaN


In [29]:
Sjarrovii = go.Scatter(x = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].year
           , y = populationSize_aut.loc[(populationSize_aut.propAutotomized.notna())].propAutotomized
          ,name = 'Autotomized <i>S. jarrovii</i>')



data = [Sjarrovii]
layout = go.Layout(
    title = 'Proportion Autotomized by Year for <i>Sceloporus jarrovii</i> adults 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18),
        range = [1999.5,2017.5]),
    yaxis = dict(
        tickformat=".2%",
        title = 'Proportion of Autotomized Lizards',
        titlefont = dict(
            size = 18),
    range=[0,1])
)
fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii adults 2000-2017 56 plus.html')
plot(fig, filename = 'Proportion Autotomized by Year for Sceloporus jarrovii adults 2000-2017 56 plus.html')

'Proportion Autotomized by Year for Sceloporus jarrovii adults 2000-2017 56 plus.html'

In [30]:
description(populationSize_aut, 'propAutotomized',True)[['mean','std','min','25%','50%','75%','max','siqr']]

mean    36.2197
std     6.69805
min     19.6078
25%     33.5821
50%     36.6601
75%     39.8361
max      46.875
siqr    3.12699
Name: propAutotomized, dtype: object

Proportion of _S. jarrovii_ with evidence of autotomy is also oscillates over the years.  
To be done:
- does this track with oscillations in the population?

### Severity of Autotomy
Here we will regress SVL and sex on TL for intact _S. jarrovii_ to estimate what the TL would have been had an animal not autotomized.

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [31]:
dfPred = df_firstInYear[['liznumber','svl','sex','tl','autotomized','year']]

In [32]:
dfPred['tl_svl'] = dfPred.tl/dfPred.svl

In [33]:
intactRatio = go.Box(y=dfPred.loc[dfPred.autotomized==False].tl_svl, name = 'Intact')
autotomizedRatio = go.Box(y=dfPred.loc[dfPred.autotomized].tl_svl, name = 'Autotmized')
data = [autotomizedRatio, intactRatio]
layout = go.Layout(
    title = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Autotomy Status',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL/SVL Ratio',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)

iplot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017 56 plus.html')
# plot(fig, filename = 'Box Plot of TL/SVL Ratio by Autotomy Status for <i>Sceloporus jarrovii<i/> CC 2000-2017 56 plus.html')

We will identify the _Sceloporus jarrovii_ adults with a tl/svl ratio outside of the center 95%.  To do this, we create a boolean column _extremeRatio_ individuals with tl/svl ratios outside of the center 95% have a value of **True** in this column.  We only use individuals without extreme ratios to model tl and svl.

In [34]:
print(dfPred.loc[dfPred.autotomized==False].tl_svl.describe())
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl>=dfPred.tl_svl.quantile(.025))&
                   (dfPred.tl_svl<=dfPred.tl_svl.quantile(.925)),'extremeRatio']= False
dfPred.loc[(dfPred.autotomized==False)&
                   (dfPred.tl_svl<dfPred.tl_svl.quantile(.025))|
                   (dfPred.tl_svl>dfPred.tl_svl.quantile(.925)),'extremeRatio']= True
dfPred.loc[dfPred.extremeRatio.isna()].autotomized.value_counts(dropna=False)


count    706.000000
mean       1.356319
std        0.101584
min        0.140845
25%        1.320000
50%        1.361757
75%        1.402933
max        1.582090
Name: tl_svl, dtype: float64


True    395
Name: autotomized, dtype: int64

In [35]:
sjM = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].tl,
                     remove_na=True)
sjM

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,3.622887,2.095079,1.729237,8.493563e-02,0.898097,0.897711,-0.502302,7.748077
1,svl,1.311695,0.027193,48.235879,6.230720e-133,0.898097,0.897711,1.258151,1.365238


In [36]:
sjF = pg.linear_regression(dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].svl,
                     dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].tl,
                     remove_na=True)
sjF

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,7.819258,2.141856,3.650693,3.010789e-04,0.811735,0.811202,3.606856,12.031661
1,svl,1.219510,0.031259,39.013040,4.672587e-130,0.811735,0.811202,1.158033,1.280987


SVL and sex are sufficient to predict TL in males.

In [37]:
femaleIntercept = [thing for thing in sjF.loc[sjF.names=='Intercept','coef']][0]
maleIntercept = [thing for thing in sjM.loc[sjM.names=='Intercept','coef']][0]
femaleSvl = [thing for thing in sjF.loc[sjF.names=='svl','coef']][0]
maleSvl = [thing for thing in sjM.loc[sjM.names=='svl','coef']][0]

print(femaleIntercept,maleIntercept,femaleSvl,maleSvl)

7.819258356097444 3.622887382596799 1.21950987076125 1.3116947113955868


In [38]:
dfPred.loc[dfPred.sex=='f','tlPred'] = dfPred.loc[dfPred.sex=='f'].svl*femaleSvl+femaleIntercept
dfPred.loc[dfPred.sex=='m','tlPred'] = dfPred.loc[dfPred.sex=='m'].svl*maleSvl+maleIntercept
dfPred['diff'] = dfPred.tlPred - dfPred.tl
dfPred['propDiff'] = dfPred['diff']/dfPred.tlPred

In [39]:
dfPred.groupby(['autotomized','sex']).diff.describe()

count       mean        std        min        25%        50%  \
autotomized sex                                                                 
False       f    389.0  -0.523241   6.439636 -14.815051  -3.668679  -0.790640   
            m    317.0  -0.761404   8.145676 -16.441536  -4.064925  -1.298819   
True        f    204.0  27.201448  16.135356   1.087400  15.697155  24.587400   
            m    191.0  30.283017  19.207286  -1.441536  17.688296  26.753212   

                       75%        max  
autotomized sex                        
False       f     1.721518  84.404459  
            m     1.623380  84.311685  
True        f    34.392277  85.502009  
            m    38.746770  95.675411

In [40]:
male = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male',mode='markers',marker=dict(size=5,opacity=0.8))
female = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female',mode='markers',marker=dict(size=5,opacity=0.8))
All = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio==False)].sort_values('svl')['tl'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
maleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='m')].sort_values('svl')['tl'],
                  name = 'Male Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
femaleOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)&(dfPred.sex=='f')].sort_values('svl')['tl'],
                    name = 'Female Extreme',mode='markers',marker=dict(size=5,opacity=0.8))
AllOut = go.Scatter(x=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['svl'],
          y=dfPred.loc[(dfPred.extremeRatio!=False)].sort_values('svl')['tl'],
                    name = 'All Extreme',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))

data = [All,male,female,AllOut,maleOut,femaleOut]
layout = go.Layout(
    title = 'Scatter Plot of SVL vs TL for Intact <i>Sceloporus jarrovii</i> adults CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        tickangle=45,
        dtick = 1,
        title = 'SVL (mm)',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
#         tickformat = ".0%",
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfPred.loc[(dfPred.autotomized==False)]['count'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii adults CC 2000-2017 56 plus.html')
plot(fig, filename = 'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii adults CC 2000-2017 56 plus.html')

'Scatter Plot of SVL vs TL for Intact Sceloporus jarrovii adults CC 2000-2017 56 plus.html'

In [41]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['autotomized','sex']).propDiff.describe()

count      mean       std       min       25%       50%  \
autotomized sex                                                            
False       f    355.0 -0.000003  0.047091 -0.073674 -0.032997 -0.005290   
            m    266.0  0.000017  0.039741 -0.060198 -0.027733 -0.005091   
True        f    204.0  0.290831  0.167776  0.012486  0.164952  0.266199   
            m    191.0  0.277994  0.171034 -0.013279  0.177235  0.243958   

                      75%       max  
autotomized sex                      
False       f    0.024008  0.407333  
            m    0.019620  0.182175  
True        f    0.368481  0.850749  
            m    0.358996  0.922390

In [42]:
dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index()

,year,autotomized,count,50%
0,2000,False,72.0,-0.001447
1,2000,True,45.0,0.219732
2,2001,False,52.0,-0.006539
3,2001,True,34.0,0.255586
4,2002,False,55.0,0.006288
5,2002,True,30.0,0.252773
6,2003,False,32.0,0.001778
7,2003,True,22.0,0.237079
8,2004,False,28.0,0.002069
9,2004,True,20.0,0.224296


In [43]:
dfYearlyEst = dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized','sex'])\
.propDiff.describe()[['count','50%']].reset_index(drop=False).merge(dfPred.loc[((dfPred.autotomized==False)&(dfPred.extremeRatio==False))|
          (dfPred.autotomized==True)].groupby(['year','autotomized'])\
.propDiff.describe()[['count','50%']].reset_index().rename(columns = {'count':'count_all',
                                                                      '50%':'50%_all'}),
                                                                    how = 'left',
                                                                    on =['year','autotomized'])
dfYearlyEst.loc[dfYearlyEst.autotomized]

,year,autotomized,sex,count,50%,count_all,50%_all
2,2000,True,f,23.0,0.218108,45.0,0.219732
3,2000,True,m,22.0,0.229342,45.0,0.219732
6,2001,True,f,11.0,0.257508,34.0,0.255586
7,2001,True,m,23.0,0.253665,34.0,0.255586
10,2002,True,f,13.0,0.172625,30.0,0.252773
11,2002,True,m,17.0,0.288258,30.0,0.252773
14,2003,True,f,13.0,0.262787,22.0,0.237079
15,2003,True,m,9.0,0.221566,22.0,0.237079
18,2004,True,f,12.0,0.123097,20.0,0.224296
19,2004,True,m,8.0,0.249314,20.0,0.224296


In [44]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')].sort_values('count_all')['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['count'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')].sort_values('count_all')['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['count_all'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)].sort_values('count_all')['50%_all'],
                    name = 'All',mode='markers',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017 56 plus.html')
plot(fig, filename = 'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017 56 plus.html')

'Scatter Plot of Population Size vs Median Proportion of Tail Loss for CC 2000-2017 56 plus.html'

In [45]:
male = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male',mode='markers')
female = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                  name = 'Female',mode='markers')
All = go.Scatter(x=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'],
          y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'],
                    name = 'All',mode='lines',marker=dict(size=20,opacity=0.5,line=dict(width=2)))
medianAll = go.Scatter(x=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['min'],
                          dfYearlyEst.loc[(dfYearlyEst.autotomized)]['year'].describe()['max']],
                      y=[dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median(),
                        dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%_all'].median()],
                         name='medianAll',mode='lines')
data = [All,male,female,medianAll]
layout = go.Layout(
    title = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
# fig.update_xaxes(range=[0,dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count_all'].describe()['max']+1])
# fig.update_yaxes(range=[0,1])

iplot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')
plot(fig, filename = 'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html')

'Scatter Plot of Median Proportion of Tail Loss for CC 2000-2017 by Year.html'

To be done:

- for each sex and all, regress prop tl on relevant count 
- add these lines to plots with CI
- report on stat (what does this tell us?)

# resume
[TOC](#Table-of-Contents)

In [46]:
male = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['50%'],
                  name = 'Male (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='m')]['count'].sum()))
female = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['50%'],
                    name = 'Female (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)&(dfYearlyEst.sex=='f')]['count'].sum()))
All = go.Box(y=dfYearlyEst.loc[(dfYearlyEst.autotomized)]['50%'],
                    name = 'All (n = {:.0f})'\
              .format(dfYearlyEst.loc[(dfYearlyEst.autotomized)]['count'].sum()))
data = [All,male,female,]
layout = go.Layout(
    title = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Population Size',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Proportion of Tail Lost',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017 56 plus.html')
plot(fig, filename = 'Boxplot Median Proportion of Tail Loss for CC 2000-2017 56 plus.html')

'Boxplot Median Proportion of Tail Loss for CC 2000-2017 56 plus.html'

In [47]:
dfPred.loc[(dfPred.autotomized==True)].groupby('sex').propDiff.describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
f,204.0,0.290831,0.167776,0.012486,0.164952,0.266199,0.368481,0.850749
m,191.0,0.277994,0.171034,-0.013279,0.177235,0.243958,0.358996,0.922390


In [48]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> adults by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii adults by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii adults by Year.html')

In [49]:
female = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')]['year'], 
                y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='f')].propDiff
                      ,name='Females')
male = go.Box(x=dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)&(dfPred.sex=='m')].propDiff
                    ,name='Males')
All = go.Box(x=dfPred.loc[(dfPred.autotomized==True)]['year'], 
              y = dfPred.loc[(dfPred.autotomized==True)].propDiff
                    ,name='All')
data = [All,male,female]
layout = go.Layout(
    title = 'Box Plot of Tail Lost at Capture for <i>Sceloporus jarrovii</i> adults by Year',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'propDiff',
        tickformat = ".0%",
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii adults by Year.html')
# plot(fig, filename = 'Box Plot of TL/SVL Sceloporus jarrovii adults by Year.html')

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Morphometrics

In this section we describe the distributions of various morphometrics.

- [SVL](#SVL)
- [TL](#TL)
- [RTL](#RTL)
- [Mass](#SVL)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

#### SVL

Now we examine the range and distribution of svl values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of SVL](#SVLhist)

In [50]:
print("svl values in the data set range from {} to {} for and are distributed across sex \
as displayed here:"\
      .format(df_firstInYear.svl.min(), df_firstInYear.svl.max()))
description(df_firstInYear.groupby('sex'),variable='svl')

svl values in the data set range from 56.0 to 98.0 for and are distributed across sex as displayed here:


,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
sex,,,,,,,,,,
f,593.0,68.762226,7.210338,56.0,63.0,68.0,75.0,86.0,6.0,not calculated
m,508.0,77.271654,9.499768,56.0,70.0,78.0,85.0,98.0,7.5,not calculated


In [51]:
SVLbyYear = description(df_firstInYear.groupby(['year','sex']),'svl').reset_index()
SVLbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,65.0,65.876923,7.507304,56.0,60.00,65.0,70.00,83.0,5.000,not calculated
1,2000,m,59.0,74.067797,10.837140,56.0,64.00,72.0,84.00,98.0,10.000,not calculated
2,2001,f,47.0,68.276596,7.288174,56.0,64.00,68.0,72.50,82.0,4.250,not calculated
3,2001,m,45.0,75.955556,10.411872,56.0,67.00,78.0,85.00,93.0,9.000,not calculated
4,2002,f,51.0,65.372549,7.541779,56.0,59.00,62.0,72.00,80.0,6.500,not calculated
5,2002,m,46.0,74.326087,10.968752,56.0,65.00,72.5,84.75,92.0,9.875,not calculated
6,2003,f,35.0,67.657143,5.900862,57.0,64.50,68.0,70.50,80.0,3.000,not calculated
7,2003,m,29.0,74.448276,10.762323,56.0,65.00,76.0,85.00,89.0,10.000,not calculated
8,2004,f,30.0,68.300000,6.752522,58.0,63.50,66.5,74.00,80.0,5.250,not calculated
9,2004,m,25.0,77.000000,8.154753,63.0,72.00,79.0,82.00,91.0,5.000,not calculated


Let's plot these values. 

##### Figures of SVL values

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)


In [52]:
female = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')].svl
                      ,name='Female Box')
male = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')].svl
                    ,name='Males Box')
female_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['50%']
                      ,name='Female Median', line = dict(color = 'red'))
male_line = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['50%']
                    ,name='Male Median', line = dict(color = 'blue'))
female_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='f')]['year'], 
                y = SVLbyYear.loc[(SVLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=SVLbyYear.loc[(SVLbyYear.sex=='m')]['year'], 
              y = SVLbyYear.loc[(SVLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [male,female,male_line,female_line,male_count,female_count]
layout = go.Layout(
    title = 'Box Plot of SVL at Capture for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median SVL (mm)',
        titlefont = dict(
            size = 18)),
    boxmode= 'group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Box Plot of Median SVL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Box Plot of Median SVL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

To be done:
- look at correlation between the count and the median svl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.
- why did the small ones show up?
    - removal of larger animals
    - increase in juveniles due to birth rates 
    - only include 56+ and rerun

#### TL

Now we examine the range and distribution of TL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of TL](#TLhist)

In [53]:
TLbyYear = description(df_firstInYear.groupby(['year','autotomized','sex']),'tl').reset_index()
TLbyYear

,year,autotomized,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,False,f,42.0,86.714286,9.796181,70.0,79.00,84.5,93.75,114.0,7.375,not calculated
1,2000,False,m,37.0,98.135135,14.171886,70.0,87.00,98.0,111.00,122.0,12.000,not calculated
2,2000,True,f,23.0,69.304348,13.039920,42.0,63.50,67.0,77.50,93.0,7.000,not calculated
3,2000,True,m,22.0,78.454545,17.081673,45.0,64.00,76.0,93.00,108.0,14.500,not calculated
4,2001,False,f,36.0,91.861111,8.626050,76.0,85.75,92.0,98.00,109.0,6.125,not calculated
5,2001,False,m,22.0,100.090909,13.309177,79.0,91.00,100.0,110.50,124.0,9.750,not calculated
6,2001,True,f,11.0,67.454545,14.989087,35.0,62.00,67.0,75.00,92.0,6.500,not calculated
7,2001,True,m,23.0,75.956522,15.399168,26.0,70.00,80.0,83.50,103.0,6.750,not calculated
8,2002,False,f,38.0,87.473684,11.074043,48.0,80.25,88.0,96.50,106.0,8.125,not calculated
9,2002,False,m,29.0,99.965517,15.760445,77.0,87.00,97.0,114.00,130.0,13.500,not calculated


##### Histogram of TL (Intact)

In [54]:
femaleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&~(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&~(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Intact Females')
maleIntact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Intact Males')
female_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[~(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxIntact,femaleBoxIntact,maleIntact,femaleIntact,male_count,female_count]
layout = go.Layout(
    title = 'TL for Intact <i>Sceloporus jarrovii</i> adults in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL for Intact Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'TL for Intact Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'TL for Intact Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

To be done:
- look at correlation between the count and the median tl 
Outliers will be addressed in the Cleaning notebook, but will be removed for the remained of the analyses here.

##### Histogram of TL (Autotomized)

In [55]:
femaleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='f')]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.sex=='f')&(df_firstInYear.autotomized)].tl
                      ,name='Females')
maleBoxAutotomized = go.Box(x=df_firstInYear.loc[(df_firstInYear.sex=='m')]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.sex=='m')&(df_firstInYear.autotomized)].tl
                    ,name='Males')
femaleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                      y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['50%']
                      ,name='Autotomized Females')
maleAutotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['50%']
                    ,name='Autotomized Males')
female_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['year'], 
                y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='f')]['count']
                      ,name='Females Count',line = dict(color = 'red',dash = 'dash'))
male_count = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['year'], 
              y = TLbyYear.loc[(TLbyYear.autotomized)&(TLbyYear.sex=='m')]['count']
                    ,name='Male Count',line = dict(color = 'blue',dash = 'dash'))

data = [maleBoxAutotomized,femaleBoxAutotomized,maleAutotomized,femaleAutotomized,male_count,female_count]
layout = go.Layout(
    title = 'TL for Autotomized <i>Sceloporus jarrovii</i> adults in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL for Autotomized Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'TL for Autotomized Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'TL for Autotomized Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

##### Histogram of TL (Intact vs. Autotomized) 2

In [56]:
TLbyYear.groupby('year')['year','50%'].apply(lambda x: x[['50%']])

,50%
0,84.5
1,98.0
2,67.0
3,76.0
4,92.0
5,100.0
6,67.0
7,80.0
8,88.0
9,97.0


In [57]:
# BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
#                 y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
#                       ,name='Autotomized')
# BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
#               y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
#                     ,name='Intact')
compared = (TLbyYear.loc[~(TLbyYear.autotomized)]['50%'].reset_index(drop=True))\
-(TLbyYear.loc[(TLbyYear.autotomized)]['50%'].reset_index(drop=True))
Comparison = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
                   y = compared,name='Intact - Autotomized',
                       mode = 'markers')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [Comparison]
layout = go.Layout(
    title = 'Comparison on Median TL of <i>Sceloporus jarrovii</i> adults in CC 2000-2017 by Tail Condition',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Comparison on Median TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Comparison on Median TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

##### Histogram of TL (Intact vs. Autotomized)

In [58]:
BoxAut = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized)]['year'], 
                y = df_firstInYear.loc[(df_firstInYear.autotomized)].tl
                      ,name='Autotomized')
BoxIntact = go.Box(x=df_firstInYear.loc[(df_firstInYear.autotomized==False)]['year'], 
              y = df_firstInYear.loc[(df_firstInYear.autotomized==False)].tl
                    ,name='Intact')
# Autotomized = go.Scatter(x=TLbyYear.loc[(TLbyYear.autotomized)]['year'], 
#                    y = TLbyYear.loc[(TLbyYear.autotomized)]['50%']
#                       ,name='Autotomized')
# Intact = go.Scatter(x=TLbyYear.loc[~(TLbyYear.autotomized)]['year'], 
#               y = TLbyYear.loc[~(TLbyYear.autotomized)]['50%']
#                     ,name='Intact')
data = [BoxAut,BoxIntact]
layout = go.Layout(
    title = 'TL of <i>Sceloporus jarrovii</i> adults in CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'TL (mm)',
        titlefont = dict(
            size = 18)),
boxmode='group')

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'TL of Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

#### RTL

Now we examine the range and distribution of RTL values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Histogram of RTL](#RTLhist)

In [59]:
RTLbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'rtl').reset_index()
RTLbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,23.0,22.826087,14.083040,0.0,13.50,27.0,32.50,48.0,9.500,not calculated
1,2000,m,22.0,22.590909,17.374472,0.0,3.25,24.5,35.25,55.0,16.000,not calculated
2,2001,f,11.0,21.181818,20.375565,0.0,0.00,23.0,38.00,50.0,19.000,not calculated
3,2001,m,23.0,19.434783,17.959110,0.0,2.00,16.0,30.50,58.0,14.250,not calculated
4,2002,f,13.0,13.000000,16.057189,0.0,1.00,5.0,23.00,44.0,11.000,not calculated
5,2002,m,17.0,27.764706,18.126394,0.0,13.00,30.0,41.00,54.0,14.000,not calculated
6,2003,f,13.0,15.461538,14.569462,1.0,4.00,9.0,32.00,39.0,14.000,not calculated
7,2003,m,9.0,25.222222,17.859016,3.0,17.00,24.0,31.00,56.0,7.000,not calculated
8,2004,f,12.0,14.416667,11.719900,0.0,7.75,11.5,17.50,36.0,4.875,not calculated
9,2004,m,8.0,25.375000,17.671102,0.0,14.75,30.0,33.50,52.0,9.375,not calculated


##### Histogram of rtl

In [60]:
femaleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='f')]['year'], 
                   y = RTLbyYear.loc[(RTLbyYear.sex=='f')]['50%']
                      ,name='Females')
maleAut = go.Scatter(x=RTLbyYear.loc[(RTLbyYear.sex=='m')]['year'], 
              y = RTLbyYear.loc[(RTLbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [maleAut,femaleAut]
layout = go.Layout(
    title = 'Median RTL for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median RTL (mm)',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median RTL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Median RTL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Median RTL Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

#### Mass

Now we examine the range and distribution of mass values by species.

[Back to Morphometrics](#Morphometrics)


We will use the [distribution](#distribution) function to do this and then plot these values.

- [Plot of mass](#Plot-of-mass)

We probably need to adjust this analysis to consider teh month in which the females were captures since gravidity or nearness to the time of parturition may bias the analysis.

In [61]:
MassbyYear = description(df_firstInYear.loc[df_firstInYear.autotomized].groupby(['year','sex']),'mass').reset_index()
MassbyYear

,year,sex,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
0,2000,f,23.0,10.308696,2.960791,6.2,8.700,10.00,11.300,17.0,1.3000,not calculated
1,2000,m,22.0,15.636364,6.756904,5.5,9.350,16.25,18.575,31.0,4.6125,not calculated
2,2001,f,11.0,10.645455,3.393925,5.0,9.450,10.80,12.050,17.7,1.3000,not calculated
3,2001,m,23.0,15.800000,5.725700,7.5,11.400,14.40,20.700,26.3,4.6500,not calculated
4,2002,f,13.0,8.753846,2.426457,5.6,6.400,9.20,10.300,13.2,1.9500,not calculated
5,2002,m,17.0,14.335294,5.509531,4.7,9.500,15.60,18.300,23.4,4.4000,not calculated
6,2003,f,6.0,10.200000,4.874013,1.5,9.700,10.50,12.575,16.0,1.4375,not calculated
7,2003,m,8.0,14.175000,5.577698,6.8,10.925,13.50,16.125,24.2,2.6000,not calculated
8,2004,f,12.0,12.025000,2.969274,6.8,9.950,12.20,14.450,16.5,2.2500,not calculated
9,2004,m,7.0,18.400000,5.276046,9.5,16.000,18.00,22.750,23.8,3.3750,not calculated


##### Plot of mass

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [62]:
female = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='f')]['year'], 
                   y = MassbyYear.loc[(MassbyYear.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=MassbyYear.loc[(MassbyYear.sex=='m')]['year'], 
              y = MassbyYear.loc[(MassbyYear.sex=='m')]['50%']
                    ,name='Males')

data = [male,female]
layout = go.Layout(
    title = 'Median Mass for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Mass (mm)',
        titlefont = dict(
            size = 18),
    range = [0,MassbyYear['50%'].max()+5]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Mass Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Median Mass Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Median Mass Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

## Captures

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.  We will use all captures for this

In [63]:
df_firstInYear.groupby('liznumber').capture.apply(lambda x: x.values)

liznumber
1                   [2]
2             [2, 1, 3]
3             [1, 2, 3]
4       [4, 2, 7, 5, 8]
5                [2, 3]
6                   [1]
8                   [1]
9                   [2]
10            [1, 3, 2]
11            [3, 2, 5]
12               [1, 2]
13               [1, 2]
14               [1, 3]
15               [1, 2]
16            [1, 3, 2]
17               [1, 2]
18                  [1]
20                  [1]
21                  [1]
22                  [1]
23                  [2]
24                  [1]
25                  [1]
26                  [1]
27                  [1]
28                  [1]
29            [5, 1, 6]
31                  [1]
32                  [1]
34                  [1]
36            [3, 2, 1]
37                  [1]
38                  [1]
39                  [1]
41                  [1]
42                  [1]
43                  [1]
44                  [1]
45                  [1]
46                  [1]
47                  [1]
49    

In [64]:
print("The maximum number of captures among lizards in the data set range from {} to {} for and are \
distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').capture.max().min(),
              df_firstInYear.groupby('liznumber').capture.max().max()))
captureMedYr = description(df_firstInYear.groupby(['year','sex']),variable='capture')[['50%','75%','max']].reset_index()
captureMedYr

The maximum number of captures among lizards in the data set range from 1 to 18 for and are distributed across species and sex as displayed here:


,year,sex,50%,75%,max
0,2000,f,1.0,1.00,3.0
1,2000,m,1.0,1.00,2.0
2,2001,f,2.0,3.00,4.0
3,2001,m,1.0,2.00,3.0
4,2002,f,2.0,3.00,5.0
5,2002,m,2.0,3.00,5.0
6,2003,f,2.0,3.00,6.0
7,2003,m,2.0,3.00,6.0
8,2004,f,1.0,1.00,2.0
9,2004,m,1.0,1.00,3.0


In [65]:
female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
                   y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
                      ,name='Females')
male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
              y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
                    ,name='Males')
# femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
#                    y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
#                       ,name='Females')
# maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
#               y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
#                     ,name='Males')

data = [male,female]#,maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18),
    range = [0,captureMedYr['50%'].max()+1]),
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Median Years Captured Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Median Years Captured  Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Median Years Captured  Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

In [66]:
# female = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='f')]['year'], 
#                    y = captureMedYr.loc[(captureMedYr.sex=='f')]['50%']
#                       ,name='Females')
# male = go.Scatter(x=captureMedYr.loc[(captureMedYr.sex=='m')]['year'], 
#               y = captureMedYr.loc[(captureMedYr.sex=='m')]['50%']
#                     ,name='Males')
femaleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='f']['year'], 
                   y = df_firstInYear.loc[df_firstInYear.sex=='f'].groupby('liznumber').capture.apply(lambda x: x.values)
                      ,name='Females')
maleBox = go.Box(x=df_firstInYear.loc[df_firstInYear.sex=='m']['year'], 
              y = df_firstInYear.loc[df_firstInYear.sex=='m'].groupby('liznumber').capture.apply(lambda x: x.values)
                    ,name='Males')

data = [maleBox,femaleBox]
layout = go.Layout(
    title = 'Median Years Captured for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Year',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Median Years Captured',
        titlefont = dict(
            size = 18))
)

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Box Years Captured Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')
plot(fig, filename = 'Box Years Captured  Sceloporus jarrovii adults in CC 2000-2017 56 plus.html')

'Box Years Captured  Sceloporus jarrovii adults in CC 2000-2017 56 plus.html'

In order to interpret these data we need to factor in person-hours for each year.

## Years

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of years over which that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of years over which each lizard was captures and finally count the number of lizards that have a given number of years over which they were captured.  We will use all captures for this.

In [67]:
(df_firstInYear.groupby('liznumber').year_diff.max()+1).value_counts(normalize=True).reset_index()\
.rename(columns={'index':'year_diff','year_diff':'proportion'}).sort_values('year_diff')

,year_diff,proportion
0,1,0.723606
1,2,0.145907
2,3,0.065243
3,4,0.027284
4,5,0.011862
6,6,0.004745
9,7,0.003559
10,8,0.002372
13,9,0.001186
12,13,0.001186


In [68]:
print("The number of years over which each lizard was captured among lizards in the data set range from {} to {}\
for and are distributed across species and sex as displayed here:"\
      .format(df_firstInYear.groupby('liznumber').year_diff.max().min()+1, 
              df_firstInYear.groupby('liznumber').year_diff.max().max()+1))
description(df_firstInYear.groupby('sex'),variable='year_diff')

The number of years over which each lizard was captured among lizards in the data set range from 1 to 17for and are distributed across species and sex as displayed here:


,count,mean,std,min,25%,50%,75%,max,siqr,meanCI
sex,,,,,,,,,,
f,593.0,0.765599,2.186575,0.0,0.0,0.0,1.0,16.0,0.5,not calculated
m,508.0,0.509843,1.360872,0.0,0.0,0.0,1.0,15.0,0.5,not calculated


In [69]:
Males = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='m')].groupby('liznumber').capture.max(),
                     name='Males')
Females = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.sex=='f')].groupby('liznumber').capture.max(),
                       name='Females')

data = [Males,Females]
layout = go.Layout(
    title = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Unique Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
iplot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017 56 plus.html')
plot(fig, filename = 'Histogram of Maximum Number of Captures by Sex for CC 2000-2017 56 plus.html')

'Histogram of Maximum Number of Captures by Sex for CC 2000-2017 56 plus.html'

### Maximum Number of Captures based on Tail condition
 - match for species, sex, size and location

In [70]:
intactFemale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                   (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                      ,name='intact females')
intactMale = go.Histogram(x = df_firstInYear.loc[(~df_firstInYear.autotomized)&
                                                 (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                      ,name='intact males')
autotomizedFemale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                        (df_firstInYear.sex=='f')].groupby('liznumber').capture.max()
                           ,name='autotomized females')
autotomizedMale = go.Histogram(x = df_firstInYear.loc[(df_firstInYear.autotomized)&
                                                      (df_firstInYear.sex=='m')].groupby('liznumber').capture.max()
                           ,name='autotomized males')

data = [intactFemale,intactMale,autotomizedFemale,autotomizedMale]
layout = go.Layout(
    title = 'Maximum Number of Captures by Tail Condition 2000-2017',
    titlefont = dict(
        size = 20),
    xaxis = dict(
        dtick = 1,
        title = 'Maximum Number of Captures',
        titlefont = dict(
            size = 18)),
    yaxis = dict(
        title = 'Number of Lizards',
        titlefont = dict(
            size = 18)))

fig = go.Figure(
        data = data,
        layout = layout)
py.iplot(fig, filename = 'Histogram of Maximum Captures by Tail Condition in Crystal Creek 2000 - 2017')

## Growth

- [SVL Growth](#SVL-Growth)
- [TL Growth](#TL-Growth)
- [RTL Growth](#RTL-Growth)
- [Mass Growth](#Mass-Growth)

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

Let's take a look at the number of times that lizards have been captured.  To do this, we will group lizards by lizard number and then look at the maximum number of captures for each lizard and finally count the number of lizards that have a given number of captures.

### SVL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate?

turn this into a function and then groupby and apply

df['svl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['svl'].max()-x['svl'].min())/(x['year_diff'].max()+1))

### TL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the tail size growth rate?

df['tl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['tl'].max()-x['tl'].min())/(x['year_diff'].max()+1))

### RTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the regrown tail size growth rate?

df['rtl_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['rtl'].max()-x['rtl'].min())/(x['year_diff'].max()+1))

### NTL Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

### Mass Growth

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

What is the body size growth rate in terms of MAss?

df['mass_growth_ann']=df.groupby('liznumber').apply(lambda x: (x['mass'].max()-x['mass'].min())/(x['year_diff'].max()+1))

tmp = df.groupby('liznumber').mass_growth_ann.nunique().reset_index()
check = tmp.loc[tmp.mass_growth_ann>1,'liznumber']
print(len(check))
check

## Correlations to Population

[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)

In [71]:
def candidate(m,dv,placement=(1,1)):
    assert(dv in m.columns)
    return m[dv].sort_values().reset_index().iloc[placement[0]:placement[1]+1,:]

from functools import reduce

def topcorr(corrdf,lowestrank,dvs):
    candidates = [candidate(corrdf,dv,(1,lowestrank)) for dv in dvs]
    merger =  reduce(lambda x, y: pd.merge(x, y, on = 'index', how = 'outer'), candidates).fillna('--')
    return merger

I need to create a df with with values to be correlated:
- count
- prop Male
- SVL
- TL
- RTL
- Mass
- TL Severity
- Sex Distr


In [72]:
dfCorrPrep = populationSize_sex.loc[populationSize_sex.sex=='f',
                                    ['year', 'sex', 'liznumber', 
                                     'liznumberYear','propFemale']]\
.merge(populationSize_aut.loc[populationSize_aut.propAutotomized.notna(),
                              ['year','liznumberYear', 'propAutotomized']], 
                         how = 'left', on = ['year','liznumberYear'])\
.merge(dfYearlyEst.loc[dfYearlyEst.autotomized,['year', 'sex', 
                                                'count_all', '50%_all']],
       on = ['year','sex'])\
.rename(columns = {'count_all':'nAutotomized', 
                   '50%_all':'medianPropTailAutotomized',
                                             'liznumber':'numberFemale'})
dfCorrPrep = dfCorrPrep[['liznumberYear', 
                         'propFemale', 'propAutotomized', 
                         'medianPropTailAutotomized']]
dfCorrPrep

,liznumberYear,propFemale,propAutotomized,medianPropTailAutotomized
0,124,0.524194,0.362903,0.219732
1,92,0.510870,0.369565,0.255586
2,97,0.525773,0.309278,0.252773
3,64,0.546875,0.343750,0.237079
4,55,0.545455,0.363636,0.224296
5,60,0.483333,0.466667,0.259661
6,60,0.516667,0.333333,0.282605
7,51,0.588235,0.196078,0.245432
8,67,0.522388,0.343284,0.284314
9,54,0.537037,0.425926,0.252428


In [73]:
#Dropping proportion of Females, but will put it back once I can order the y-axis
dfCorr = dfCorrPrep.corr()
dfCorr = topcorr(dfCorr,2,dfCorr.columns.tolist()) 
dfCorr = dfCorr.set_index('index')
dfCorr

,liznumberYear,propFemale,propAutotomized,medianPropTailAutotomized
index,,,,
propAutotomized,-0.148286,--,--,0.029951
medianPropTailAutotomized,-0.104901,-0.0153272,0.029951,--
liznumberYear,--,-0.334651,-0.148286,--
propFemale,--,--,--,-0.0153272


In [74]:
testx = dfCorr.columns
testy = dfCorr.index
testz = dfCorr.values
test = go.Figure(go.Heatmap(x=testx,y=testy,z=testz))
plot(test, filename = 'population correlation matrix.html')
iplot(test, filename = 'population correlation matrix.html')

## Export Files
[Back to Top](#Table-of-Contents)

[Back to Analyze Data](#Analyze-Data)